Esta parte es para empezar el entrenamiento, poner todas al mismo tamanio las fotos
normalizar (valores entre 0 y 1) esto sirve para que esten todos los datos en la misma escala (sino los modelos en redes neuronales son chotos para aprender si hay valores de entrada muy diferentes, tipo outliers)

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Ver que hay en el drive

In [ ]:
!ls '/content/drive/My Drive/'

 Classroom					  Informe.gdoc
'Colab Notebooks'				  Main.java
'Documento sin título.gdoc'			  Proyecto-DataSets
'F1Quiz&Learn SRS.docx'				  trabajo-practico-1-gonzalez-testa-gastaldi
'F1Quiz&Learn SRS.gdoc'				  trabajo-practico-2-centeno-campregher-gonzalez
'Informe de Pruebas y Cobertura de Código.gdoc'


In [9]:
import os
import pandas as pd

# Ruta de la carpeta de imágenes y del CSV en Google Drive
image_dir = '/content/drive/My Drive/Proyecto-DataSets/imagenes_claras'
csv_file = '/content/drive/My Drive/Proyecto-DataSets/dataset.csv'

# Cargar el CSV
df = pd.read_csv(csv_file)
df['image'] = df['image'].astype(str).fillna('')
# Listas para almacenar imágenes válidas y filas válidas
valid_images = []
valid_rows = []

# Verificar la existencia de cada imagen en el CSV
for i, row in df.iterrows():
    image_path = os.path.join(image_dir, row['image'])
    if os.path.exists(image_path):
        valid_images.append(row['image'])
        valid_rows.append(row)

# Crear un nuevo DataFrame con solo las filas válidas
df_clean = pd.DataFrame(valid_rows)

# Guardar el nuevo CSV limpio en Google Drive
clean_csv_path = '/content/drive/My Drive/Proyecto-DataSets/dataset_clean.csv'
df_clean.to_csv(clean_csv_path, index=False)

print(f"Total de imágenes válidas: {len(valid_images)}")
print(f"Nuevo CSV generado en: {clean_csv_path}")


Total de imágenes válidas: 2596
Nuevo CSV generado en: /content/drive/My Drive/Proyecto-DataSets/dataset_clean.csv


In [31]:
import tensorflow as tf
import pandas as pd # pandas es para manejar el .csv labels y eso, de manera eficiente.
import os
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = '/content/drive/My Drive/Proyecto-DataSets/imagenes_claras'
csv_file = '/content/drive/My Drive/Proyecto-DataSets/dataset_clean.csv'
df = pd.read_csv(csv_file)

datagen = ImageDataGenerator(
    rescale=1./255, # normalizacion los valores de píxel entre 0 y 1
    validation_split=0.2 # % 80 training, 20 % test
)

df['image'] = df['image'].astype(str)
df['label'] = df['label'].astype(str)

# training
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=dataset_path,
    x_col="image",
    y_col="label",
    target_size=(224, 224),  # estandar
    class_mode="categorical",  # clasificación multiclase
    batch_size=32,  # tamaño de batch
    subset="training",  # usar esta parte para el entrenamiento
    shuffle=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# validacion/test
validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=dataset_path,
    x_col="image",
    y_col="label",
    target_size=(224, 224), # estandar
    class_mode="categorical", # clasificación multiclase
    batch_size=32,  # tamaño de batch
    subset="validation",  # Usar esta parte para validación
    shuffle=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


# Calcular pesos de clase para el balanceo. (20 fotos de un ingrediente y 4 del otro)
class_labels = df['label'].astype(str).values
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weight_dict = dict(enumerate(class_weights))


Found 2077 validated image filenames belonging to 109 classes.
Found 519 validated image filenames belonging to 109 classes.


Definicion del modelo y configuracion para tenerlo listo para entrenar

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential([
    # Conv2d filtro convolucional a la imagen.
    # es el area de la imagen que se procesa a la vez el primer numero, lo otro 3x3 es agrupacion que toma.
    # cada vez mas pasa por distintos filtros y mas tamanio.
    # va agarrando las caracteristicas mas importantes de las imagenes (textura, bordes, patrones).

    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),

    # MaxPooling2d agrupacion
    # 2x2 es decir una tablita y agrupa para agarrar los datos mas relevantes de cada parte de la matriz original.
    # elige el valor maximo.

    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # otra capa si hace falta con mas
    # Conv2D(256, (3, 3), activation='relu'),
    # MaxPooling2D(pool_size=(2, 2)),

    # aplanamiento
    # para que se convierta en 1d ya que las capas que vienen estan en 1d
    Flatten(),

    # aniadiendo capas densas
    # esto sirve para que la red conecte todo lo extraido del filtrado y la agrupacion)
    # en base a esto se genera una prediccion con estos datos
    # ver cuanto le ponemos a Dense
    Dense(300, activation='relu'), # cantidad de neuronas a usar.
    Dropout(0.6),
    Dense(len(train_generator.class_indices), activation='softmax') # softmax se usa para clasificacion multiclase, convierte valores en probabilidades,
                                                                     # la suma total es 1 y cada neurona de la capa densa representa
                                                                     # la probabilidad de que toque una clase
])


# compilacion
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# un resumen si se quiere
# model.summary()

# Early stopping si fuera necesario
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
# Configurar reducción de la tasa de aprendizaje cuando la pérdida de validación se estabiliza
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# checkpoint = tf.keras.callbacks.ModelCheckpoint(
#    'best_model.keras',
#    monitor='val_loss',
#    save_best_only=True
# )


Entrenamiento

In [39]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,
    class_weight=class_weight_dict,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 59s 797ms/step - accuracy: 0.0153 - loss: 5.1991 - val_accuracy: 0.0000e+00 - val_loss: 4.7714
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0000e+00 - loss: 3.8698 - val_accuracy: 0.0000e+00 - val_loss: 4.7856
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 616ms/step - accuracy: 0.0385 - loss: 5.4105 - val_accuracy: 0.0000e+00 - val_loss: 5.0067
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step - accuracy: 0.0625 - loss: 3.5006 - val_accuracy: 0.0000e+00 - val_loss: 5.0631
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 613ms/step - accuracy: 0.0455 - loss: 4.5898 - val_accuracy: 0.0000e+00 - val_loss: 5.2979
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.0625 - loss: 4.4133 - val_accuracy: 0.0000e+00 - val_loss: 5.3057
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 640ms/step - accuracy: 0.0608 - loss: 4.7570 - val_accuracy: 0.0020 - val_loss: 5.6567
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step - accuracy: 0.0625 - l

In [40]:
# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Pérdida en validación: {val_loss}")
print(f"Precisión en validación: {val_accuracy}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 514ms/step - accuracy: 0.0000e+00 - loss: 4.7727
Pérdida en validación: 4.771616458892822
Precisión en validación: 0.0


In [ ]:
ver si estan todas las fotos validas para entrenar.

In [ ]:
from PIL import Image
import os

def verify_images(image_dir):
    valid_images = []
    invalid_images = []

    for img_file in os.listdir(image_dir):
        try:
            img = Image.open(os.path.join(image_dir, img_file))
            img.verify()  # Verifica si el archivo es una imagen válida
            valid_images.append(img_file)
        except (IOError, SyntaxError) as e:
            print(f"Imagen inválida o corrupta: {img_file}")
            invalid_images.append(img_file)

    return valid_images, invalid_images

valid_images, invalid_images = verify_images('imagenes_claras')

print(f"Imágenes válidas: {len(valid_images)}")
print(f"Imágenes inválidas: {len(invalid_images)}")


Imágenes válidas: 2605
Imágenes inválidas: 0


Descarga del modelo entrenado

In [42]:
model_save_path = '/content/drive/My Drive/Proyecto-DataSets/modeloEntrenado.keras'
model.save(model_save_path)


KeyboardInterrupt: 